In [1]:
# για το scraping
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
# σύνδεση με το gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# βασικό url του site
main_url = 'https://www.efsyn.gr'
# αρχικό url των teaser pages
start_url = 'https://www.efsyn.gr/ekloges-2023?page='

In [4]:
# scraping των teasers pages για να μαζέψω τα urls των άρθρων
teasers_list = []
start_page = 1
end_page = 30

for page_num in range(start_page, end_page+1):
  page_url = start_url + str(page_num)
  response = requests.get(page_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  teaser_articles_list = doc.find('div', {'class': 'default-teaser triple'}).find_all('article')

  for teaser in teaser_articles_list:
      story_dict = {}
      try:
        link = teaser.find('a', {'class': 'full-link'})['href']
        if link.startswith("http"):
          story_dict['url'] = link
        else:
          story_dict['url'] = main_url + link
      except:
        story_dict['url'] = ''
        pass
      teasers_list.append(story_dict)

efsyn_teasers_df = pd.DataFrame(teasers_list)

In [5]:
# scraping των full_articles
full_articles_list = []

for article_url in efsyn_teasers_df['url']:
  response = requests.get(article_url)
  doc = BeautifulSoup(response.text, 'html.parser')
  article = doc.find('section', {'class': 'article__main'})
  full_article_dict = {}

  # website
  full_article_dict['site'] = "efsyn.gr"

  # url
  full_article_dict['url'] = article_url

  #section
  try:
    section = doc.find('a', {'class': 'article__category'}).text
    full_article_dict['section'] = section
  except:
    full_article_dict['section'] = ''
    pass

  #title
  try:
    title = title = article.find('h1').text
    full_article_dict['title'] = title
  except:
    full_article_dict['title'] = ''
    pass

  # date
  try:
    date = article.find('time')['datetime']
    full_article_dict['date'] = date
  except:
    full_article_dict['date'] = ''
    pass

  # author
  try:
    author = article.find('div', {'class', "article__author"}).span.text
    full_article_dict['author'] = author
  except:
    full_article_dict['author'] = ''
    pass

  # full_text
  try:
    p_texts_list = []
    paragraphs = article.find_all('p')
    for p in paragraphs:
      p_texts_list.append(p.text)
    full_text = ' '.join(p_texts_list)
    full_text = "".join(full_text.splitlines())
    full_article_dict['full_text'] = full_text
  except:
    full_article_dict['full_text'] = ''
    pass

  full_articles_list.append(full_article_dict)
efsyn_full_articles_df = pd.DataFrame(full_articles_list)
efsyn_full_articles_df


,site,url,section,title,date,author,full_text
0,efsyn.gr,https://www.efsyn.gr/stiles/ypografoyn/396587_...,ΥΠΟΓΡΑΦΟΥΝ,Ο χρόνος τρέχει κι αυτοί... χτενίζονται!,2023-07-06 16:00,Τάσος Παππάς,Η επόμενη εκλογική σύγκρουση είναι αυτή των πε...
1,efsyn.gr,https://www.efsyn.gr/stiles/apopseis/396454_ta...,ΑΠΟΨΕΙΣ,Τα πολιτικά απόνερα των οικονομικών κρίσεων,2023-07-05 16:00,Γιάννης Σιώτος,"Το 2015 οι ακαδημαϊκοί Manuel Funke, ερευνητής..."
2,efsyn.gr,https://www.efsyn.gr/stiles/apopseis/396457_ti...,ΑΠΟΨΕΙΣ,"Τίμιοι, ανδρείοι αλλά και άμπαλοι;",2023-07-05 16:00,Αντώνης Τελόπουλος,"Σε ανύποπτο χρόνο, ένα από τα πιο προβεβλημένα..."
3,efsyn.gr,https://www.efsyn.gr/politiki/396405_pos-psifi...,ΠΟΛΙΤΙΚΗ,Πώς ψήφισαν οι νέοι στις εκλογές,2023-07-05 06:00,Δημήτρης Κουκλουμπέρης,"Την ψήφο των νέων (17-24, φοιτητές και 25-34 ε..."
4,efsyn.gr,https://www.efsyn.gr/politiki/396323_ekloges-2...,ΠΟΛΙΤΙΚΗ,Εκλογές 2023: Η ψήφος των νέων από τον Μάιο στ...,2023-07-04 16:01,efsyn.gr,Συγκριτική ανάλυση του Eteron - Ινστιτούτο για...
...,...,...,...,...,...,...,...
355,efsyn.gr,https://www.efsyn.gr/stiles/triti-matia/393978...,ΤΡΙΤΗ ΜΑΤΙΑ,Ο ΣΥΡΙΖΑ ως «Κατέχων»,2023-06-16 16:00,Γιάννος Θανασέκος*,"Στο σημερινό κλίμα έντασης, αβεβαιότητας και ά..."
356,efsyn.gr,https://www.efsyn.gr/politiki/394027_bale-sti-...,ΠΟΛΙΤΙΚΗ,"«Βάλε στη Βουλή τις ελπίδες σου, όχι τους φόβο...",2023-06-16 15:32,efsyn.gr,Η συμμαχία Πράσινο & Μωβ ανακοίνωσε επίσημα τη...
357,efsyn.gr,https://www.efsyn.gr/politiki/ekloges/394024_a...,ΕΚΛΟΓΕΣ,"«Ανείπωτη τραγωδία« από τη μία η Πύλος, υπέρ τ...",2023-06-16 15:18,efsyn.gr,«Ανείπωτη τραγωδία τεραστίων διαστάσεων» χαρακ...
358,efsyn.gr,https://www.efsyn.gr/politiki/393999_syriza-ka...,ΠΟΛΙΤΙΚΗ,ΣΥΡΙΖΑ κατά Ν.Δ.: Εκφοβίζουν και τρομοκρατούν ...,2023-06-16 13:08,efsyn.gr,Κλιμακώνεται και η πολιτική σύγκρουση στη σκιά...


In [6]:
efsyn_full_articles_df.to_csv('drive/MyDrive/efsyn.csv', index=False)